 1. IMPORTAMOS LAS LIBRESIAS


In [ ]:
import pandas as pd
import numpy as np
import ast
import json

In [ ]:
# Se realiza la carga de los datasets movie y credits
data_movie = pd.read_csv("Dataset\movies_dataset.csv")
data_credits = pd.read_csv("Dataset\credits.csv")



2. COMPRENDEMOS LOS DATOS

In [ ]:

# Revisamos el numero total de registros de los datasets
print(data_movie.shape)
print(data_credits.shape)


In [ ]:
# checkeamos los tipos de datos de ambos datasets
data_movie.info()
data_credits.info()

3. PREPARACION DE LOS DATOS

In [ ]:
# se miran los nulos por columnas
data_movie.isnull().sum()

In [ ]:
#Para ahora rellenar los valores nulos en revenue y budget
data_movie['revenue'] = data_movie['revenue'].fillna(0)
data_movie['budget'] = data_movie['budget'].fillna(0)

In [ ]:
# se eliminará valores nulos de release date
data_movie.dropna(subset=['release_date'], inplace=True)


In [ ]:
# Se guardan solo los registron que no tengan release_date nula
data_movie = data_movie[data_movie['release_date'].notna()]

In [ ]:
# Revisamos otra vez los nulos despues de los cambios realizados
data_movie.isna().sum()

In [ ]:
# Se hace el cambio de formato de fecha a AAAA-mm-dd
data_movie['release_date'] = pd.to_datetime(data_movie['release_date'], format='%Y-%m-%d', errors='coerce')

In [ ]:
# Se crea una nueva columna 'release_year', donde se extrae el año de estreno
data_movie['release_year'] = data_movie['release_date'].dt.year
data_movie['release_year']

In [ ]:
# se borran los valores no validos en budget
data_movie = data_movie[pd.to_numeric(data_movie['budget'], errors='coerce').notnull()]

In [ ]:
# se convierte budget y revenue a tipo de dato float
data_movie['budget'] = data_movie['budget'].astype(np.float64)
data_movie['revenue'] = data_movie['revenue'].astype(np.float64)

In [ ]:
# Se realiza el calculo del retorno en nueva columna
data_movie['return'] = data_movie.apply(lambda x: x['revenue'] / x['budget'] if x['budget'] != 0 else 0, axis=1)

In [ ]:
# cambiamos el tipo de datos de popularity
data_movie['popularity'] = data_movie['popularity'].astype('float')

In [ ]:
# Examinamos los repetidos para su posterior eliminacion
print(data_movie.duplicated().sum())
print(data_credits.duplicated().sum())

In [ ]:
# Eliminamos duplicados
data_movie = data_movie.drop_duplicates()
data_credits = data_credits.drop_duplicates()

In [ ]:
# Se requiere revisar los duplicados en 'id' 
print(data_movie['id'].duplicated().sum())
print(data_credits['id'].duplicated().sum())



In [ ]:
# Elimino los duplicados en 'id' 
data_movie = data_movie.drop_duplicates(subset = 'id')
data_credits = data_credits.drop_duplicates(subset='id')

In [ ]:
# Se uniran los dos datasets para conformar uno solo 
# se nesecita cambiar el tipo de dato para su posterior union
data_movie['id'] = data_movie['id'].astype('int64')

In [ ]:
#Se eliminan columnas que no serán de uso
data_movie.drop(columns=['video','imdb_id','adult','original_title','poster_path','homepage'], inplace = True)


In [ ]:
# se establece la union de los dataset movie y credits para un solo dataset
d_concat = data_movie.merge(data_credits, on='id', how = 'inner')
d_concat

In [ ]:
# El dataset nos muestra que tiene campos anidados, se revisara como estan compuestos 
# para su posterior desanidado
print(d_concat['belongs_to_collection'][0])
print(d_concat['genres'][0])
print(d_concat['production_companies'][0])
print(d_concat['production_countries'][0])
print(d_concat['spoken_languages'][0])
print(d_concat['cast'][0])
print(d_concat['crew'][0])

In [ ]:
# Desanidamos la columna 'belongs_to_collection'
d_concat['belongs_to_collection'] = d_concat['belongs_to_collection'].apply(lambda x:ast.literal_eval(x)['name'] if isinstance(x,str) else'').tolist()
d_concat

In [ ]:
# extraemos los generos especificos de las peliculas en el campo 'genres'
d_concat['genres']= d_concat['genres'].str.replace("'",'"')
d_concat['genres'] = d_concat['genres'].apply(lambda x:[item['name'] for item in json.loads(x)]).tolist()
d_concat.head(3)

In [ ]:
# Creamos una funcionn para desanidar 
# los campos 'Production_companies','Production_countries' y 'spoken_languages'
def desanidar(a):
    if pd.isna(a):
        return none
    else:
        b = ast.literal_eval(a)
        if isinstance(b,list):
            return [i['name'] for i in b]
            return none

In [ ]:
#Aplicamos la funcion 'desanidar' en 'Production_companies'
d_concat['production_companies'] = d_concat['production_companies'].apply(desanidar)
d_concat['production_companies'].head(3)

In [ ]:
#Aplicamos la funcion 'desanidar' en 'Production_countries'
d_concat['production_countries'] = d_concat['production_countries'].apply(desanidar)
d_concat['production_countries'].head(3)

In [ ]:
#Aplicamos la funcion 'desanidar' a 'spoken_languages'
d_concat['spoken_languages'] = d_concat['spoken_languages'].apply(desanidar)
d_concat['spoken_languages'].head(3)

In [ ]:
#Aplicamos la funcion 'desanidar' a 'cast'
d_concat['cast'] = d_concat['cast'].apply(desanidar)
d_concat['cast'].head(3)

In [ ]:
#Para el campo 'crew' requerimos solo el nombre del director, y en este esta referencia en job 
# se nesecita crear una nueva funcion para extraer el nombre
def desanidar_crew(a):
    b = ast.literal_eval(a)
    for i in b:
        if i.get('job') == 'Director':
            return i.get('name')
    return None


In [36]:
# Aplicamos la funcion 'desanidar_crew' a 'crew' y ademas se crea un nuevo campo 'director'
d_concat['director'] = d_concat['crew'].apply(desanidar_crew) 
d_concat.head(3)

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,...,status,tagline,title,vote_average,vote_count,release_year,return,cast,crew,director
0,Toy Story Collection,30000000.0,"[Animation, Comedy, Family]",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,[Pixar Animation Studios],[United States of America],1995-10-30,...,Released,NaN,Toy Story,7.7,5415.0,1995.0,12.451801,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",John Lasseter
1,,65000000.0,"[Adventure, Fantasy, Family]",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[TriStar Pictures, Teitler Film, Interscope Co...",[United States of America],1995-12-15,...,Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,1995.0,4.043035,"[Robin Williams, Jonathan Hyde, Kirsten Dunst,...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",Joe Johnston
2,Grumpy Old Men Collection,0.0,"[Romance, Comedy]",15602,en,A family wedding reignites the ancient feud be...,11.712900,"[Warner Bros., Lancaster Gate]",[United States of America],1995-12-22,...,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,1995.0,0.000000,"[Walter Matthau, Jack Lemmon, Ann-Margret, Sop...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",Howard Deutch


In [51]:
#Creo un nuevo data set solo con los campos requeridos para realizar unas funciones
# especificas para este proyecto
Dataset_Final = d_concat[['id','title','genres','director','belongs_to_collection','production_companies','production_countries',
                 'original_language','release_date','release_year','runtime','vote_average',
                 'vote_count','popularity','revenue','budget','return']] 

In [52]:
Dataset_Final.head(3)

,id,title,genres,director,belongs_to_collection,production_companies,production_countries,original_language,release_date,release_year,runtime,vote_average,vote_count,popularity,revenue,budget,return
0,862,Toy Story,"[Animation, Comedy, Family]",John Lasseter,Toy Story Collection,[Pixar Animation Studios],[United States of America],en,1995-10-30,1995.0,81.0,7.7,5415.0,21.946943,373554033.0,30000000.0,12.451801
1,8844,Jumanji,"[Adventure, Fantasy, Family]",Joe Johnston,,"[TriStar Pictures, Teitler Film, Interscope Co...",[United States of America],en,1995-12-15,1995.0,104.0,6.9,2413.0,17.015539,262797249.0,65000000.0,4.043035
2,15602,Grumpier Old Men,"[Romance, Comedy]",Howard Deutch,Grumpy Old Men Collection,"[Warner Bros., Lancaster Gate]",[United States of America],en,1995-12-22,1995.0,101.0,6.5,92.0,11.712900,0.0,0.0,0.000000


In [54]:
# Listo tenemos nuestro de dasetfinal, exportamos el archivo a .csv para trabajar las funciones
Dataset_Final.to_csv(r"C:\Users\angel\Desktop\RATG\Datasciencie\Henry-Proyect\Dataset_Final.csv")

In [40]:
d_concat.to_csv(r"C:\Users\angel\Desktop\RATG\Datasciencie\Nuevo Proyecto\Proyecto-Individual\dataset\Dataset_PI.csv")